In [2]:
import pandas as pd
import numpy as np
import re
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
# ===== Bagian B: Load Dataset dan Kamus Slang / Stopword =====
df = pd.read_csv("Ulasan_Penelitian_Fixkali  - Worksheet.csv")
# Mengatur tampilan agar tidak memotong isi kolom
pd.set_option('display.max_colwidth', None)

slang_dict = dict(line.strip().split(':') for line in open("kamus_slang.txt", encoding='utf-8'))
stopword_custom = set(open("stopwordsID.txt", encoding='utf-8').read().splitlines())

In [3]:
# ===== Bagian C: Preprocessing =====
print("\n=== Preprocessing Data ===")
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def print_step(step_name, before, after):
    """Utility untuk menampilkan before-after"""
    print(f"\n=== {step_name.upper()} ===")
    print(f"Before: {before}")
    print(f"After:  {after}")


=== Preprocessing Data ===


In [4]:
# === Langkah 1: Case Folding + Phrase Standardization ===
def case_folding(text):
    before = text
    after = text.lower()
    after = re.sub(r'\bgo[\s\-_]?ride\b', 'goride', after)
    
    # Normalisasi istilah umum: "go ride", "go-ride", dll → "goride"
    # after = re.sub(r'\bgo[\s\-_]?ride\b', 'goride', after)
    # # after = re.sub(r'\bgo[\s\-_]?jek\b', 'gojek', after)  # tambahan jika ada
    # # after = re.sub(r'\bgrab[\s\-_]?bike\b', 'grabbike', after)  # opsional
    print_step("Case Folding + Standardization", before, after)
    return after

df['casefolding'] = df['ulasan'].astype(str).apply(case_folding)

# Menampilkan hasil 5 data pertama
print("\n=== DataFrame setelah Case Folding ===")
df[['ulasan', 'casefolding']].rename(columns={
    'ulasan': 'sebelum case folding', 
    'casefolding': 'setelah case folding'
}).head(5)


=== CASE FOLDING + STANDARDIZATION ===
Before: Aneh, voucher goride gocar gamuncul semua padahal saya punya banyak voucher yang muncul cuma voucher gosend diperbaiki deh tolong
After:  aneh, voucher goride gocar gamuncul semua padahal saya punya banyak voucher yang muncul cuma voucher gosend diperbaiki deh tolong

=== CASE FOLDING + STANDARDIZATION ===
Before: Kok buka aplikasi goride hilang2 mulu...
After:  kok buka aplikasi goride hilang2 mulu...

=== CASE FOLDING + STANDARDIZATION ===
Before: Gojek keren bisa pesen sekalian 2 goride satu waktu .... makasih gojek 😘😍🥰
After:  gojek keren bisa pesen sekalian 2 goride satu waktu .... makasih gojek 😘😍🥰

=== CASE FOLDING + STANDARDIZATION ===
Before: Sekarang udah mahal goride dan gofood
After:  sekarang udah mahal goride dan gofood

=== CASE FOLDING + STANDARDIZATION ===
Before: Kesalahan dari sistem gojek saldo Gopay kepotong otomatis ketika melakukan transaksi order GoRide GoRide nya BATAL tp saldo Gopay kepotong dan gak balik lagi. U

,sebelum case folding,setelah case folding
0,"Aneh, voucher goride gocar gamuncul semua padahal saya punya banyak voucher yang muncul cuma voucher gosend diperbaiki deh tolong","aneh, voucher goride gocar gamuncul semua padahal saya punya banyak voucher yang muncul cuma voucher gosend diperbaiki deh tolong"
1,Kok buka aplikasi goride hilang2 mulu...,kok buka aplikasi goride hilang2 mulu...
2,Gojek keren bisa pesen sekalian 2 goride satu waktu .... makasih gojek 😘😍🥰,gojek keren bisa pesen sekalian 2 goride satu waktu .... makasih gojek 😘😍🥰
3,Sekarang udah mahal goride dan gofood,sekarang udah mahal goride dan gofood
4,Kesalahan dari sistem gojek saldo Gopay kepotong otomatis ketika melakukan transaksi order GoRide GoRide nya BATAL tp saldo Gopay kepotong dan gak balik lagi. Udh bikin laporan dan disuruh tuunggu 2x24 jam tp saldo ga masuk lagi ke akun gue. Hadeuuu rumit,kesalahan dari sistem gojek saldo gopay kepotong otomatis ketika melakukan transaksi order goride goride nya batal tp saldo gopay kepotong dan gak balik lagi. udh bikin laporan dan disuruh tuunggu 2x24 jam tp saldo ga masuk lagi ke akun gue. hadeuuu rumit


In [5]:
# === Langkah 2: Cleansing ===
def clean_text(text):
    before = text
    after = re.sub(r'http\S+|www\S+', '', text)
    after = re.sub(r'[^a-zA-Z\s]', ' ', after)
    after = re.sub(r'\s+', ' ', after).strip()
    print_step("Cleansing", before, after)
    return after

df['cleansing'] = df['casefolding'].apply(clean_text)
print("\n=== DataFrame setelah Cleansing ===")
df[['casefolding', 'cleansing']].rename(columns={'casefolding': 'sebelum cleansing', 'cleansing': 'setelah cleansing'}).head(5)


=== CLEANSING ===
Before: aneh, voucher goride gocar gamuncul semua padahal saya punya banyak voucher yang muncul cuma voucher gosend diperbaiki deh tolong
After:  aneh voucher goride gocar gamuncul semua padahal saya punya banyak voucher yang muncul cuma voucher gosend diperbaiki deh tolong

=== CLEANSING ===
Before: kok buka aplikasi goride hilang2 mulu...
After:  kok buka aplikasi goride hilang mulu

=== CLEANSING ===
Before: gojek keren bisa pesen sekalian 2 goride satu waktu .... makasih gojek 😘😍🥰
After:  gojek keren bisa pesen sekalian goride satu waktu makasih gojek

=== CLEANSING ===
Before: sekarang udah mahal goride dan gofood
After:  sekarang udah mahal goride dan gofood

=== CLEANSING ===
Before: kesalahan dari sistem gojek saldo gopay kepotong otomatis ketika melakukan transaksi order goride goride nya batal tp saldo gopay kepotong dan gak balik lagi. udh bikin laporan dan disuruh tuunggu 2x24 jam tp saldo ga masuk lagi ke akun gue. hadeuuu rumit
After:  kesalahan dari si

,sebelum cleansing,setelah cleansing
0,"aneh, voucher goride gocar gamuncul semua padahal saya punya banyak voucher yang muncul cuma voucher gosend diperbaiki deh tolong",aneh voucher goride gocar gamuncul semua padahal saya punya banyak voucher yang muncul cuma voucher gosend diperbaiki deh tolong
1,kok buka aplikasi goride hilang2 mulu...,kok buka aplikasi goride hilang mulu
2,gojek keren bisa pesen sekalian 2 goride satu waktu .... makasih gojek 😘😍🥰,gojek keren bisa pesen sekalian goride satu waktu makasih gojek
3,sekarang udah mahal goride dan gofood,sekarang udah mahal goride dan gofood
4,kesalahan dari sistem gojek saldo gopay kepotong otomatis ketika melakukan transaksi order goride goride nya batal tp saldo gopay kepotong dan gak balik lagi. udh bikin laporan dan disuruh tuunggu 2x24 jam tp saldo ga masuk lagi ke akun gue. hadeuuu rumit,kesalahan dari sistem gojek saldo gopay kepotong otomatis ketika melakukan transaksi order goride goride nya batal tp saldo gopay kepotong dan gak balik lagi udh bikin laporan dan disuruh tuunggu x jam tp saldo ga masuk lagi ke akun gue hadeuuu rumit


In [6]:
# === Langkah 3: Normalisasi Slang ===
def normalize_slang(text):
    before = text
    words = text.split()
    normalized = [slang_dict.get(word, word) for word in words]
    after = ' '.join(normalized)
    print_step("Normalize Slang", before, after)
    return after

df['normalized'] = df['cleansing'].apply(normalize_slang)
print("\n=== DataFrame setelah Normalize Slang ===")
df[['cleansing', 'normalized']].rename(columns={'cleansing': 'sebelum normalize slang', 'normalized': 'setelah normalize slang'}).head(5)


=== NORMALIZE SLANG ===
Before: aneh voucher goride gocar gamuncul semua padahal saya punya banyak voucher yang muncul cuma voucher gosend diperbaiki deh tolong
After:  aneh voucher goride gocar tidak muncul semua padahal saya punya banyak voucher yang muncul hanya voucher gosend diperbaiki deh tolong

=== NORMALIZE SLANG ===
Before: kok buka aplikasi goride hilang mulu
After:  kenapa buka aplikasi goride hilang melulu

=== NORMALIZE SLANG ===
Before: gojek keren bisa pesen sekalian goride satu waktu makasih gojek
After:  gojek keren bisa pesan sekalian goride satu waktu terima kasih gojek

=== NORMALIZE SLANG ===
Before: sekarang udah mahal goride dan gofood
After:  sekarang sudah mahal goride dan gofood

=== NORMALIZE SLANG ===
Before: kesalahan dari sistem gojek saldo gopay kepotong otomatis ketika melakukan transaksi order goride goride nya batal tp saldo gopay kepotong dan gak balik lagi udh bikin laporan dan disuruh tuunggu x jam tp saldo ga masuk lagi ke akun gue hadeuuu rumit


,sebelum normalize slang,setelah normalize slang
0,aneh voucher goride gocar gamuncul semua padahal saya punya banyak voucher yang muncul cuma voucher gosend diperbaiki deh tolong,aneh voucher goride gocar tidak muncul semua padahal saya punya banyak voucher yang muncul hanya voucher gosend diperbaiki deh tolong
1,kok buka aplikasi goride hilang mulu,kenapa buka aplikasi goride hilang melulu
2,gojek keren bisa pesen sekalian goride satu waktu makasih gojek,gojek keren bisa pesan sekalian goride satu waktu terima kasih gojek
3,sekarang udah mahal goride dan gofood,sekarang sudah mahal goride dan gofood
4,kesalahan dari sistem gojek saldo gopay kepotong otomatis ketika melakukan transaksi order goride goride nya batal tp saldo gopay kepotong dan gak balik lagi udh bikin laporan dan disuruh tuunggu x jam tp saldo ga masuk lagi ke akun gue hadeuuu rumit,kesalahan dari sistem gojek saldo gopay kepotong otomatis ketika melakukan transaksi order goride goride nya batal tapi saldo gopay kepotong dan tidak balik lagi sudah bikin laporan dan disuruh tuunggu x jam tapi saldo tidak masuk lagi ke akun saya aduh rumit


In [7]:
# === Langkah 4: Remove Repeated Characters ===
def remove_repeated(text):
    before = text
    after = re.sub(r'(\w)\1{2,}', r'\1\1', text)
    print_step("Remove Repeated Characters", before, after)
    return after

df['no_repeated'] = df['normalized'].apply(remove_repeated)
print("\n=== DataFrame setelah Remove Repeated Characters ===")
df[['normalized', 'no_repeated']].rename(columns={'normalized': 'sebelum remove repeated', 'no_repeated': 'setelah remove repeated'}).head(5)


=== REMOVE REPEATED CHARACTERS ===
Before: aneh voucher goride gocar tidak muncul semua padahal saya punya banyak voucher yang muncul hanya voucher gosend diperbaiki deh tolong
After:  aneh voucher goride gocar tidak muncul semua padahal saya punya banyak voucher yang muncul hanya voucher gosend diperbaiki deh tolong

=== REMOVE REPEATED CHARACTERS ===
Before: kenapa buka aplikasi goride hilang melulu
After:  kenapa buka aplikasi goride hilang melulu

=== REMOVE REPEATED CHARACTERS ===
Before: gojek keren bisa pesan sekalian goride satu waktu terima kasih gojek
After:  gojek keren bisa pesan sekalian goride satu waktu terima kasih gojek

=== REMOVE REPEATED CHARACTERS ===
Before: sekarang sudah mahal goride dan gofood
After:  sekarang sudah mahal goride dan gofood

=== REMOVE REPEATED CHARACTERS ===
Before: kesalahan dari sistem gojek saldo gopay kepotong otomatis ketika melakukan transaksi order goride goride nya batal tapi saldo gopay kepotong dan tidak balik lagi sudah bikin lapora

,sebelum remove repeated,setelah remove repeated
0,aneh voucher goride gocar tidak muncul semua padahal saya punya banyak voucher yang muncul hanya voucher gosend diperbaiki deh tolong,aneh voucher goride gocar tidak muncul semua padahal saya punya banyak voucher yang muncul hanya voucher gosend diperbaiki deh tolong
1,kenapa buka aplikasi goride hilang melulu,kenapa buka aplikasi goride hilang melulu
2,gojek keren bisa pesan sekalian goride satu waktu terima kasih gojek,gojek keren bisa pesan sekalian goride satu waktu terima kasih gojek
3,sekarang sudah mahal goride dan gofood,sekarang sudah mahal goride dan gofood
4,kesalahan dari sistem gojek saldo gopay kepotong otomatis ketika melakukan transaksi order goride goride nya batal tapi saldo gopay kepotong dan tidak balik lagi sudah bikin laporan dan disuruh tuunggu x jam tapi saldo tidak masuk lagi ke akun saya aduh rumit,kesalahan dari sistem gojek saldo gopay kepotong otomatis ketika melakukan transaksi order goride goride nya batal tapi saldo gopay kepotong dan tidak balik lagi sudah bikin laporan dan disuruh tuunggu x jam tapi saldo tidak masuk lagi ke akun saya aduh rumit


In [8]:
# === Langkah 5: Tokenization ===
def tokenize_text(text):
    before = text
    after = re.findall(r'\b\w+\b', text)
    print_step("Tokenization", before, after)
    return after

df['tokenized'] = df['no_repeated'].apply(tokenize_text)
print("\n=== DataFrame setelah Tokenization ===")
# Pakai repr() agar elemen list terlihat sebagai string Python valid
df_token_display = df[['no_repeated', 'tokenized']].copy()
df_token_display['tokenized'] = df_token_display['tokenized'].apply(repr)
df_token_display.rename(columns={'no_repeated': 'sebelum tokenization', 'tokenized': 'setelah tokenization'}).head(5)


=== TOKENIZATION ===
Before: aneh voucher goride gocar tidak muncul semua padahal saya punya banyak voucher yang muncul hanya voucher gosend diperbaiki deh tolong
After:  ['aneh', 'voucher', 'goride', 'gocar', 'tidak', 'muncul', 'semua', 'padahal', 'saya', 'punya', 'banyak', 'voucher', 'yang', 'muncul', 'hanya', 'voucher', 'gosend', 'diperbaiki', 'deh', 'tolong']

=== TOKENIZATION ===
Before: kenapa buka aplikasi goride hilang melulu
After:  ['kenapa', 'buka', 'aplikasi', 'goride', 'hilang', 'melulu']

=== TOKENIZATION ===
Before: gojek keren bisa pesan sekalian goride satu waktu terima kasih gojek
After:  ['gojek', 'keren', 'bisa', 'pesan', 'sekalian', 'goride', 'satu', 'waktu', 'terima', 'kasih', 'gojek']

=== TOKENIZATION ===
Before: sekarang sudah mahal goride dan gofood
After:  ['sekarang', 'sudah', 'mahal', 'goride', 'dan', 'gofood']

=== TOKENIZATION ===
Before: kesalahan dari sistem gojek saldo gopay kepotong otomatis ketika melakukan transaksi order goride goride nya batal ta

,sebelum tokenization,setelah tokenization
0,aneh voucher goride gocar tidak muncul semua padahal saya punya banyak voucher yang muncul hanya voucher gosend diperbaiki deh tolong,"['aneh', 'voucher', 'goride', 'gocar', 'tidak', 'muncul', 'semua', 'padahal', 'saya', 'punya', 'banyak', 'voucher', 'yang', 'muncul', 'hanya', 'voucher', 'gosend', 'diperbaiki', 'deh', 'tolong']"
1,kenapa buka aplikasi goride hilang melulu,"['kenapa', 'buka', 'aplikasi', 'goride', 'hilang', 'melulu']"
2,gojek keren bisa pesan sekalian goride satu waktu terima kasih gojek,"['gojek', 'keren', 'bisa', 'pesan', 'sekalian', 'goride', 'satu', 'waktu', 'terima', 'kasih', 'gojek']"
3,sekarang sudah mahal goride dan gofood,"['sekarang', 'sudah', 'mahal', 'goride', 'dan', 'gofood']"
4,kesalahan dari sistem gojek saldo gopay kepotong otomatis ketika melakukan transaksi order goride goride nya batal tapi saldo gopay kepotong dan tidak balik lagi sudah bikin laporan dan disuruh tuunggu x jam tapi saldo tidak masuk lagi ke akun saya aduh rumit,"['kesalahan', 'dari', 'sistem', 'gojek', 'saldo', 'gopay', 'kepotong', 'otomatis', 'ketika', 'melakukan', 'transaksi', 'order', 'goride', 'goride', 'nya', 'batal', 'tapi', 'saldo', 'gopay', 'kepotong', 'dan', 'tidak', 'balik', 'lagi', 'sudah', 'bikin', 'laporan', 'dan', 'disuruh', 'tuunggu', 'x', 'jam', 'tapi', 'saldo', 'tidak', 'masuk', 'lagi', 'ke', 'akun', 'saya', 'aduh', 'rumit']"


In [9]:
# === Langkah 6: Stopword Removal ===
def remove_stopwords(tokens):
    if isinstance(tokens, str):
        tokens = re.findall(r'\b\w+\b', tokens)
    before = tokens.copy()
    after = [word for word in tokens if word not in stopword_custom]
    print_step("Stopword Removal", before, after)
    return after

# Terapkan fungsi ke DataFrame
df['no_stopword'] = df['tokenized'].apply(remove_stopwords)

print("\n=== DataFrame setelah Stopword Removal ===")
df_stopword_display = df[['tokenized', 'no_stopword']].copy()
df_stopword_display['tokenized'] = df_stopword_display['tokenized'].apply(repr)
df_stopword_display['no_stopword'] = df_stopword_display['no_stopword'].apply(repr)

# Rename dan tampilkan 10 baris pertama
df_stopword_display.rename(columns={
    'tokenized': 'sebelum no_stopword',
    'no_stopword': 'setelah no_stopword'
}).head(5)


=== STOPWORD REMOVAL ===
Before: ['aneh', 'voucher', 'goride', 'gocar', 'tidak', 'muncul', 'semua', 'padahal', 'saya', 'punya', 'banyak', 'voucher', 'yang', 'muncul', 'hanya', 'voucher', 'gosend', 'diperbaiki', 'deh', 'tolong']
After:  ['aneh', 'voucher', 'goride', 'tidak', 'muncul', 'voucher', 'muncul', 'voucher', 'diperbaiki', 'tolong']

=== STOPWORD REMOVAL ===
Before: ['kenapa', 'buka', 'aplikasi', 'goride', 'hilang', 'melulu']
After:  ['buka', 'aplikasi', 'goride', 'hilang', 'melulu']

=== STOPWORD REMOVAL ===
Before: ['gojek', 'keren', 'bisa', 'pesan', 'sekalian', 'goride', 'satu', 'waktu', 'terima', 'kasih', 'gojek']
After:  ['gojek', 'keren', 'bisa', 'pesan', 'goride', 'terima', 'kasih', 'gojek']

=== STOPWORD REMOVAL ===
Before: ['sekarang', 'sudah', 'mahal', 'goride', 'dan', 'gofood']
After:  ['mahal', 'goride']

=== STOPWORD REMOVAL ===
Before: ['kesalahan', 'dari', 'sistem', 'gojek', 'saldo', 'gopay', 'kepotong', 'otomatis', 'ketika', 'melakukan', 'transaksi', 'order', 'go

,sebelum no_stopword,setelah no_stopword
0,"['aneh', 'voucher', 'goride', 'gocar', 'tidak', 'muncul', 'semua', 'padahal', 'saya', 'punya', 'banyak', 'voucher', 'yang', 'muncul', 'hanya', 'voucher', 'gosend', 'diperbaiki', 'deh', 'tolong']","['aneh', 'voucher', 'goride', 'tidak', 'muncul', 'voucher', 'muncul', 'voucher', 'diperbaiki', 'tolong']"
1,"['kenapa', 'buka', 'aplikasi', 'goride', 'hilang', 'melulu']","['buka', 'aplikasi', 'goride', 'hilang', 'melulu']"
2,"['gojek', 'keren', 'bisa', 'pesan', 'sekalian', 'goride', 'satu', 'waktu', 'terima', 'kasih', 'gojek']","['gojek', 'keren', 'bisa', 'pesan', 'goride', 'terima', 'kasih', 'gojek']"
3,"['sekarang', 'sudah', 'mahal', 'goride', 'dan', 'gofood']","['mahal', 'goride']"
4,"['kesalahan', 'dari', 'sistem', 'gojek', 'saldo', 'gopay', 'kepotong', 'otomatis', 'ketika', 'melakukan', 'transaksi', 'order', 'goride', 'goride', 'nya', 'batal', 'tapi', 'saldo', 'gopay', 'kepotong', 'dan', 'tidak', 'balik', 'lagi', 'sudah', 'bikin', 'laporan', 'dan', 'disuruh', 'tuunggu', 'x', 'jam', 'tapi', 'saldo', 'tidak', 'masuk', 'lagi', 'ke', 'akun', 'saya', 'aduh', 'rumit']","['kesalahan', 'sistem', 'gojek', 'saldo', 'gopay', 'kepotong', 'otomatis', 'transaksi', 'order', 'goride', 'goride', 'nya', 'batal', 'saldo', 'gopay', 'kepotong', 'tidak', 'bikin', 'laporan', 'disuruh', 'tuunggu', 'x', 'jam', 'saldo', 'tidak', 'masuk', 'akun', 'aduh', 'rumit']"


In [10]:
# === Langkah 7: Stemming ===
def stem_tokens(tokens):
    if isinstance(tokens, str):
        tokens = tokens.split()
    before = tokens.copy()
    after = [stemmer.stem(word) for word in tokens]
    print_step("Stemming", before, after)
    return after

# Terapkan proses stemming ke kolom 'no_stopword'
df['stemmed'] = df['no_stopword'].apply(stem_tokens)

print("\n=== DataFrame setelah Stemming ===")

# Format tampilan agar sama seperti langkah stopword removal
df_stemming_display = df[['no_stopword', 'stemmed']].copy()
df_stemming_display['no_stopword'] = df_stemming_display['no_stopword'].apply(repr)
df_stemming_display['stemmed'] = df_stemming_display['stemmed'].apply(repr)

# Rename kolom dan tampilkan 10 baris pertama
df_stemming_display = df_stemming_display.rename(columns={
    'no_stopword': 'sebelum stemming',
    'stemmed': 'setelah stemming'
})
df_stemming_display.head(5)


=== STEMMING ===
Before: ['aneh', 'voucher', 'goride', 'tidak', 'muncul', 'voucher', 'muncul', 'voucher', 'diperbaiki', 'tolong']
After:  ['aneh', 'voucher', 'goride', 'tidak', 'muncul', 'voucher', 'muncul', 'voucher', 'baik', 'tolong']

=== STEMMING ===
Before: ['buka', 'aplikasi', 'goride', 'hilang', 'melulu']
After:  ['buka', 'aplikasi', 'goride', 'hilang', 'melulu']

=== STEMMING ===
Before: ['gojek', 'keren', 'bisa', 'pesan', 'goride', 'terima', 'kasih', 'gojek']
After:  ['gojek', 'keren', 'bisa', 'pesan', 'goride', 'terima', 'kasih', 'gojek']

=== STEMMING ===
Before: ['mahal', 'goride']
After:  ['mahal', 'goride']

=== STEMMING ===
Before: ['kesalahan', 'sistem', 'gojek', 'saldo', 'gopay', 'kepotong', 'otomatis', 'transaksi', 'order', 'goride', 'goride', 'nya', 'batal', 'saldo', 'gopay', 'kepotong', 'tidak', 'bikin', 'laporan', 'disuruh', 'tuunggu', 'x', 'jam', 'saldo', 'tidak', 'masuk', 'akun', 'aduh', 'rumit']
After:  ['salah', 'sistem', 'gojek', 'saldo', 'gopay', 'potong', '

,sebelum stemming,setelah stemming
0,"['aneh', 'voucher', 'goride', 'tidak', 'muncul', 'voucher', 'muncul', 'voucher', 'diperbaiki', 'tolong']","['aneh', 'voucher', 'goride', 'tidak', 'muncul', 'voucher', 'muncul', 'voucher', 'baik', 'tolong']"
1,"['buka', 'aplikasi', 'goride', 'hilang', 'melulu']","['buka', 'aplikasi', 'goride', 'hilang', 'melulu']"
2,"['gojek', 'keren', 'bisa', 'pesan', 'goride', 'terima', 'kasih', 'gojek']","['gojek', 'keren', 'bisa', 'pesan', 'goride', 'terima', 'kasih', 'gojek']"
3,"['mahal', 'goride']","['mahal', 'goride']"
4,"['kesalahan', 'sistem', 'gojek', 'saldo', 'gopay', 'kepotong', 'otomatis', 'transaksi', 'order', 'goride', 'goride', 'nya', 'batal', 'saldo', 'gopay', 'kepotong', 'tidak', 'bikin', 'laporan', 'disuruh', 'tuunggu', 'x', 'jam', 'saldo', 'tidak', 'masuk', 'akun', 'aduh', 'rumit']","['salah', 'sistem', 'gojek', 'saldo', 'gopay', 'potong', 'otomatis', 'transaksi', 'order', 'goride', 'goride', 'nya', 'batal', 'saldo', 'gopay', 'potong', 'tidak', 'bikin', 'lapor', 'suruh', 'tuunggu', 'x', 'jam', 'saldo', 'tidak', 'masuk', 'akun', 'aduh', 'rumit']"


In [11]:
# === Langkah 8: Rejoin Tokens ===
def rejoin_tokens(tokens):
    if isinstance(tokens, str):
        return tokens
    before = tokens.copy()
    after = ' '.join(tokens)
    print_step("Rejoin Tokens", before, after)
    return after

df['final_clean'] = df['stemmed'].apply(rejoin_tokens)
print("\n=== DataFrame setelah Final Clean ===")
df[['stemmed', 'final_clean']].rename(columns={'stemmed': 'sebelum rejoin tokens', 'final_clean': 'setelah rejoin tokens'}).head(10)


=== REJOIN TOKENS ===
Before: ['aneh', 'voucher', 'goride', 'tidak', 'muncul', 'voucher', 'muncul', 'voucher', 'baik', 'tolong']
After:  aneh voucher goride tidak muncul voucher muncul voucher baik tolong

=== REJOIN TOKENS ===
Before: ['buka', 'aplikasi', 'goride', 'hilang', 'melulu']
After:  buka aplikasi goride hilang melulu

=== REJOIN TOKENS ===
Before: ['gojek', 'keren', 'bisa', 'pesan', 'goride', 'terima', 'kasih', 'gojek']
After:  gojek keren bisa pesan goride terima kasih gojek

=== REJOIN TOKENS ===
Before: ['mahal', 'goride']
After:  mahal goride

=== REJOIN TOKENS ===
Before: ['salah', 'sistem', 'gojek', 'saldo', 'gopay', 'potong', 'otomatis', 'transaksi', 'order', 'goride', 'goride', 'nya', 'batal', 'saldo', 'gopay', 'potong', 'tidak', 'bikin', 'lapor', 'suruh', 'tuunggu', 'x', 'jam', 'saldo', 'tidak', 'masuk', 'akun', 'aduh', 'rumit']
After:  salah sistem gojek saldo gopay potong otomatis transaksi order goride goride nya batal saldo gopay potong tidak bikin lapor suruh 

,sebelum rejoin tokens,setelah rejoin tokens
0,"[aneh, voucher, goride, tidak, muncul, voucher, muncul, voucher, baik, tolong]",aneh voucher goride tidak muncul voucher muncul voucher baik tolong
1,"[buka, aplikasi, goride, hilang, melulu]",buka aplikasi goride hilang melulu
2,"[gojek, keren, bisa, pesan, goride, terima, kasih, gojek]",gojek keren bisa pesan goride terima kasih gojek
3,"[mahal, goride]",mahal goride
4,"[salah, sistem, gojek, saldo, gopay, potong, otomatis, transaksi, order, goride, goride, nya, batal, saldo, gopay, potong, tidak, bikin, lapor, suruh, tuunggu, x, jam, saldo, tidak, masuk, akun, aduh, rumit]",salah sistem gojek saldo gopay potong otomatis transaksi order goride goride nya batal saldo gopay potong tidak bikin lapor suruh tuunggu x jam saldo tidak masuk akun aduh rumit
5,"[gojek, bagus, lebih, banyakin, promo, nya, pesan, goride, jarak, nya, banget, kena, harga, tara, sma, jarak]",gojek bagus lebih banyakin promo nya pesan goride jarak nya banget kena harga tara sma jarak
6,"[overall, bantu, gera, goride, driver, tidak, ambil, pilih, pakai, goride, hemat, tambah, voucher, ringan, sekali, sering, andal, bisnis, one, system, tidak, pilih, pilh, kayak, customer, neng, driver, tidak, ambil]",overall bantu gera goride driver tidak ambil pilih pakai goride hemat tambah voucher ringan sekali sering andal bisnis one system tidak pilih pilh kayak customer neng driver tidak ambil
7,"[pesan, goride, titik, lokasi, sulit, temu, driver, pdahal, klau, pesan, pakai, lokasi, tera, titik, kordinat]",pesan goride titik lokasi sulit temu driver pdahal klau pesan pakai lokasi tera titik kordinat
8,"[sangat, ganggu, iklan, timbul, aplikasi, jek, hape, tulis, pesan, telpon, iklan, muncul, kerja, kendala, sangat, bantu, gojek, baik, goride, mart]",sangat ganggu iklan timbul aplikasi jek hape tulis pesan telpon iklan muncul kerja kendala sangat bantu gojek baik goride mart
9,"[goridenya, bagus]",goridenya bagus


In [12]:
# Ganti nama kolom 'final_clean' menjadi 'clean_text'
df['clean_text'] = df['final_clean']

# Load ulang data asli agar tidak membawa semua kolom preprocessing
df_asli = pd.read_csv("Ulasan_Penelitian_Fixkali  - Worksheet.csv")

# Gabungkan kolom 'clean_text' ke data asli
df_final = pd.concat([df_asli, df['clean_text']], axis=1)

# Simpan ke file baru agar tidak menimpa yang lama
df_final.to_csv("Ulasan_Penelitian_Fixkali_Cleaned.csv", index=False, encoding='utf-8-sig')

print("\n✅ File dengan kolom 'clean_text' berhasil disimpan sebagai 'Ulasan_Penelitian_Fixkali_Cleaned.csv'")



✅ File dengan kolom 'clean_text' berhasil disimpan sebagai 'Ulasan_Penelitian_Fixkali_Cleaned.csv'


In [6]:
df[['ulasan', 'ulasan_bersih']].rename(columns={'ulasan': 'sebelum preprocessing', 'ulasan_bersih': 'setelah preprocessing'}).head(10)

,sebelum preprocessing,setelah preprocessing
0,"Aneh, voucher goride gocar gamuncul semua padahal saya punya banyak voucher yang muncul cuma voucher gosend diperbaiki deh tolong",aneh voucher goride tidak muncul voucher muncul voucher baik tolong
1,Kok buka aplikasi goride hilang2 mulu...,buka aplikasi goride hilang melulu
2,Gojek keren bisa pesen sekalian 2 goride satu waktu .... makasih gojek 😘😍🥰,gojek keren bisa pesan goride terima kasih gojek
3,Sekarang udah mahal goride dan gofood,mahal goride
4,Kesalahan dari sistem gojek saldo Gopay kepotong otomatis ketika melakukan transaksi order GoRide GoRide nya BATAL tp saldo Gopay kepotong dan gak balik lagi. Udh bikin laporan dan disuruh tuunggu 2x24 jam tp saldo ga masuk lagi ke akun gue. Hadeuuu rumit,salah sistem gojek saldo gopay potong otomatis transaksi order goride goride nya batal saldo gopay potong tidak bikin lapor suruh tuunggu x jam saldo tidak masuk akun aduh rumit
5,"Gojek udh bagus sih, cuma lebih banyakin lagi promo nya, trus juga kalo pesen goride padahal jarak nya deket bgt tpi kenapa hrus kena harga yang setara sma jarak jauh.",gojek bagus lebih banyakin promo nya pesan goride jarak nya banget kena harga tara sma jarak
6,Overall membantu pergerakan saya lewat Goride atau GoCar. Hanya saja driver pasti tidak mau ambil kalo pilih yang pake Gocar atau Goride hemat. Padahal dengan tambahan voucher jadi meringankan sekali. Bagi saya yang sering juga mengandalkan Gosend untuk bisnis. Jadi percuma. Sebaiknya one system aja gak usah pilih pilh kayak gitu. Customer udah seneng tapi driver ndak ada yang mau ambil.,overall bantu gera goride driver tidak ambil pilih pakai goride hemat tambah voucher ringan sekali sering andal bisnis one system tidak pilih pilh kayak customer neng driver tidak ambil
7,"Setiap pesan goride atau gofood selalu aja titik lokasi saya sulit ditemukan oleh driver, pdahal klau pesan memakai lokasi yg tertera dg titik kordinat",pesan goride titik lokasi sulit temu driver pdahal klau pesan pakai lokasi tera titik kordinat
8,"saya sangat terganggu dengan iklan yg timbul dari aplikasi GO-JEK di hape bahkan saat saya sedang menulis pesan atau sedang menelpon tiba-tiba iklan muncul sehingga pekerjaan saya jadi terkendala. Padahal sebelumnya saya sangat terbantu dengan gojek baik untuk go ride, go food atau go Mart.",sangat ganggu iklan timbul aplikasi jek hape tulis pesan telpon iklan muncul kerja kendala sangat bantu gojek baik goride mart
9,Goridenya bagus,goridenya bagus


In [ ]:
# STEP 1: Import library
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

# STEP 2: Load IndoBERT model dan tokenizer
model_name = "mdhugol/indonesia-bert-sentiment-classification"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# STEP 3: Load dataset CSV
df = pd.read_csv("goride_review.csv")
print(f"Jumlah ulasan: {len(df)}")
df.head()

# STEP 4: Buat pipeline analisis sentimen
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# STEP 5: Lakukan pseudo-labeling (berlabel otomatis)
# Proses batch untuk menghindari error terlalu banyak permintaan
batch_size = 100
labels = []

for i in range(0, len(df), batch_size):
    batch = df["review_text"].iloc[i:i+batch_size].tolist()
    preds = sentiment_pipeline(batch)
    labels.extend([pred["label"] for pred in preds])

df["pseudo_label"] = labels

In [9]:
import pandas as pd

# Load Dataset
df = pd.read_csv("Ulasan_Penelitian_Fixkali_Cleaned.csv")

# Rename column and display the head
df[['ulasan_bersih', 'sentiment']].rename(columns={'ulasan_bersih': 'ulasan'}).head(5)

,ulasan,sentiment
0,aneh voucher goride tidak muncul voucher muncu...,Negative
1,buka aplikasi goride hilang melulu,Negative
2,gojek keren bisa pesan goride terima kasih gojek,Positive
3,mahal goride,Negative
4,salah sistem gojek saldo gopay potong otomatis...,Negative
